# **COLONIAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and 'alfa' in nombre and "puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
    #and ("jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_q_puebla',
 'df_alfa_septiembre_2024_puebla']

In [33]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','Colonia':'colonia'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [34]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio','colonia']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Más baratas

In [35]:
df_concatenado = pd.concat(dataframes_list)
    # Normalizar el nombre de la colonia
df_concatenado['colonia'] = df_concatenado['colonia'].str.lower().str.strip()

# Reemplazar después de la normalización
df_concatenado['colonia'] = df_concatenado['colonia'].replace('uni?n popular', 'unión popular')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('uni??n Popular', 'unión popular')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('uni??n popular', 'unión popular')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('lomas de nueva airosa.', 'lomas de nueva airosa')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('josǹ l??pez portillo', 'josé lópez portillo')

df_concatenado['Colonia'] = df_concatenado['Colonia'].replace('uni?n popular', 'unión popular')
df_concatenado['Colonia'] = df_concatenado['Colonia'].replace('uni??n Popular', 'unión popular')
df_concatenado['Colonia'] = df_concatenado['Colonia'].replace('uni??n popular', 'unión popular')
df_concatenado['Colonia'] = df_concatenado['Colonia'].replace('lomas de nueva airosa.', 'lomas de nueva airosa')
df_concatenado['Colonia'] = df_concatenado['Colonia'].replace('josǹ l??pez portillo', 'josé lópez portillo')

    # Agrupar por colonia y calcular el promedio de precios
df_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()
    # Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
df_promedio_colonia['colonia'] = df_promedio_colonia['colonia'].str.title()
    # Ordenar por precio promedio en orden ascendente
df_promedio_colonia = df_promedio_colonia.sort_values(by='precio')

# Seleccionar las colonias más baratas
top_colonias_baratas = df_promedio_colonia.head(5)
top_colonias_baratas

KeyError: 'Colonia'

In [36]:
colores = ['#df00ff','#939fad','#6773df','#67c2df','#7cc668','#ffb038','#ff4853','#3d45c0']
col = top_colonias_baratas['colonia']
precio = top_colonias_baratas['precio']
fig = go.Figure()
fig.add_trace(go.Bar(
    y=col[::-1],  # Invertir el orden de los datos
    x=precio[::-1],  # Invertir el orden de los datos
    orientation='h', 
    marker_color='#34a853',  # Colores de las barras
    text=precio[::-1].apply(lambda x: '${:,.2f}'.format(x)),  # Invertir el orden de los datos
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Colonias más baratas (Tulancingo)',  # Título de la gráfica
    xaxis=dict(
        title='$', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje X
        range=[0, 600000],  # Establecer el rango del eje y
        tickvals=[0, 200000,400000,600000],  # Definir los valores de las marcas en el eje y
        #ticktext=['$0.00', '$200,000.00', '$4,000,000.00','$6,000,000.00','$8,000,000.00'],  # Definir el texto de las marcas en el eje y
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_colonias_baratas', carpeta='assets/graficas')
fig.show()

## Más caras

In [37]:
# Reemplazar "felipe Ãngeles" por "felipe angeles" después de la normalización
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinconada la concepci??n', 'rinconada la concepción')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('riinc?n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag?n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag??n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc?n de la plata', 'rincón de la plata')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc??n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('revoluci?n', 'revolución')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('parque urbanosan antonio el desmonte', 'parque urbano san antonio el desmonte')


In [38]:
# Agrupar por colonia y calcular el promedio de precios
df_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()
# Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
df_promedio_colonia['colonia'] = df_promedio_colonia['colonia'].str.title()
# Ordenar por precio promedio en orden descendente
df_promedio_colonia = df_promedio_colonia.sort_values(by='precio', ascending=False)

# Seleccionar las colonias más caras
top_colonias_caras = df_promedio_colonia.head(5)
top_colonias_caras

,colonia,precio
334,"Fraccionamiento Galaxia La Laguna, Puebla, Puebla",25600000.0
811,"Tepetzintla, Puebla, Puebla",24700000.0
244,"El Riego Sur, Puebla, Puebla",24000000.0
501,"Las Animas, Puebla De Zaragoza, Puebla",21500000.0
162,"Condominio Residencial Palmas, Puebla, Puebla",21000000.0


In [40]:
colores = ['#df00ff','#939fad','#6773df','#67c2df','#7cc668','#ffb038','#ff4853','#3d45c0']
col = top_colonias_caras['colonia']
precio = top_colonias_caras['precio']
fig = go.Figure()
fig.add_trace(go.Bar(
    y=col[::-1],  # Invertir el orden de los datos
    x=precio[::-1],  # Invertir el orden de los datos
    orientation='h', 
    marker_color='#ea4335',  # Colores de las barras
    text=precio[::-1].apply(lambda x: '${:,.2f}'.format(x)),  # Invertir el orden de los datos
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Colonias más caras (Tulancingo)',  # Título de la gráfica
    xaxis=dict(
        title='$', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje X
        range=[0, 26000000],  # Establecer el rango del eje y
        #tickvals=[0, 2000000,4000000,6000000, 8000000],  # Definir los valores de las marcas en el eje y
        #ticktext=['$0.00', '$2,000,000.00', '$4,000,000.00','$6,000,000.00','$8,000,000.00'],  # Definir el texto de las marcas en el eje y
    ),   margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_colonias_caras', carpeta='assets/graficas')
fig.show()

## Más predominantes

In [41]:
# Reemplazar "felipe Ãngeles" por "felipe angeles" después de la normalización
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe ãngeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('felipe angeles', 'felipe ángeles')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinconada la concepci??n', 'rinconada la concepción')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('riinc?n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag?n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('ricardo flores mag??n', 'ricardo flores magón')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc?n de la plata', 'rincón de la plata')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('rinc??n del valle', 'rincón del valle')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('revoluci?n', 'revolución')
df_concatenado['colonia'] = df_concatenado['colonia'].replace('parque urbanosan antonio el desmonte', 'parque urbano san antonio el desmonte')


In [42]:
# Calcular la frecuencia de cada colonia
frecuencia_colonia = df_concatenado['colonia'].value_counts().reset_index()
frecuencia_colonia.columns = ['colonia', 'total']
# Eliminar la entrada 'No obtenida' si está presente
frecuencia_colonia = frecuencia_colonia[frecuencia_colonia['colonia'] != 'No obtenida']
frecuencia_colonia = frecuencia_colonia[frecuencia_colonia['colonia'] != 'no obtenida']

# Calcular el total de ocurrencias en el DataFrame original
total_ocurrencias = frecuencia_colonia['total'].sum()

# Calcular el porcentaje para cada colonia
frecuencia_colonia['porcentaje'] = (frecuencia_colonia['total'] / total_ocurrencias) * 100

# Seleccionar el top 5 de colonias que más se repiten
top_5_colonias = frecuencia_colonia.head(5)

# Calcular el precio promedio de cada colonia
precio_promedio_colonia = df_concatenado.groupby('colonia')['precio'].mean().reset_index()

# Fusionar los DataFrames de frecuencia y precio promedio por colonia
colonias_predominantes = pd.merge(top_5_colonias, precio_promedio_colonia, on='colonia', how='left')
# Convertir la primera letra de cada palabra en la columna "colonia" a mayúscula
colonias_predominantes['colonia'] = colonias_predominantes['colonia'].str.title()
colonias_predominantes

,colonia,total,porcentaje,precio
0,Centro,443,5.566026,3.710110e+06
1,Puebla,329,4.133685,3.646831e+06
2,La Paz,256,3.216484,4.644100e+06
3,Santa Cruz Buenavista,177,2.223897,3.252047e+06
4,Jardines De San Manuel,139,1.746451,9.294604e+05


In [43]:
fig = go.Figure()
porcentajes = colonias_predominantes['porcentaje']
colonias = colonias_predominantes['colonia']
# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=colonias,  # Eje x: Modalidades
    y=porcentajes,  # Eje y: Porcentajes
     marker_color='#9500ff',   # Especifica los colores de las barras
    text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
    texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Colonias Predominantes en Tulancingo',  # Título de la gráfica
    #xaxis=dict(title='%'),  # Título del eje x
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        #gridwidth=1,
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom) 
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_colonias_predominantes', carpeta='assets/graficas')
fig.show()